<a href="https://colab.research.google.com/github/dylanscoggins/CSCE4205Project/blob/main/Group0Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project Imports

In [2]:
import pandas as pd

#Data Sets

In [3]:
# download data sets from GitHub repository
pokemonDataSet01 = 'https://raw.githubusercontent.com/dylanscoggins/CSCE4205Project/main/PokemonDataSet01.csv'
ds1 = pd.read_csv(pokemonDataSet01)

pokemonDataSet02 = 'https://raw.githubusercontent.com/dylanscoggins/CSCE4205Project/main/PokemonDataSet02.csv'
ds2 = pd.read_csv(pokemonDataSet02)

In [ ]:
ds1

In [ ]:
ds2